In [1]:
import torch
from scipy.io.wavfile import write
from mel_processing import spectrogram_torch
from text import text_to_sequence, _clean_text
from models import SynthesizerTrn
import utils
import commons
from torch import no_grad, LongTensor
import logging

logging.getLogger('numba').setLevel(logging.WARNING)

C:\Users\ljj66\miniconda3\envs\vits\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 设置参数
model_path = "models/Umamusume/uma_1153000.pth"
config_path = "models/Umamusume/uma87.json"
text_to_read = "私たちは、オペラを作ってみましょう！　艶のあるチョコレート ケーキで、トレセン学園の『重厚さ』が表現できると思うんです！"
speaker_id = 29
output_path = ""
cleaned = False
length_scale = 1.0
noise_scale = 0.667
noise_scale_w = 0.8

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hps_ms = utils.get_hparams_from_file(config_path)
n_symbols = len(hps_ms.symbols) if 'symbols' in hps_ms.keys() else 0
n_speakers = hps_ms.data.n_speakers if 'n_speakers' in hps_ms.data.keys() else 0
speakers = hps_ms.speakers if 'speakers' in hps_ms.keys() else ['0']

net_g_ms = SynthesizerTrn(
    n_symbols,
    hps_ms.data.filter_length // 2 + 1,
    hps_ms.train.segment_size // hps_ms.data.hop_length,
    n_speakers=n_speakers,
    **hps_ms.model).to(device)
_ = net_g_ms.eval()
utils.load_checkpoint(model_path, net_g_ms)

def get_text(text, hps, cleaned=False):
    if cleaned:
        text_norm = text_to_sequence(text, hps.symbols, [])
    else:
        text_norm = text_to_sequence(text, hps.symbols, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = LongTensor(text_norm)
    return text_norm

stn_tst = get_text(text_to_read, hps_ms, cleaned=cleaned)
with no_grad():
    x_tst = stn_tst.unsqueeze(0).to(device)
    x_tst_lengths = LongTensor([stn_tst.size(0)]).to(device)
    sid = LongTensor([speaker_id]).to(device)
    audio = net_g_ms.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=noise_scale,
                           noise_scale_w=noise_scale_w, length_scale=length_scale)[0][0, 0].data.cpu().float().numpy()

write(output_path, hps_ms.data.sampling_rate, audio)
print('Successfully saved!')

Successfully saved!
